In [ ]:
%load_ext jupyter_black

In [ ]:
from datetime import datetime, timedelta
from IPython.display import display, HTML
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz
import random
import threading
from time import sleep, time

from utils import (
    get_quantiles,
    plot_line,
    plot_quantiles,
)

In [ ]:
# Constants

In [ ]:
DATA_DIR = "../data/"
LOG_DIR = "../logs/"

REFRESH_SEC = 1

FILENAME = ""
START_DATETIME = ""
END_DATETIME = ""

In [ ]:
DATE_ENTRY_LABEL_WIDTH = 100
DATE_ENTRY_WIDTH = 400

TOKEN_ENTRY_LABEL_WIDTH = 100
TOKEN_ENTRY_WIDTH = 225
REFRESH_BUTTON_WIDTH = 225

LOG_WIDTH = 500

SUMMARY_LABEL_SIZE = 15
SUMMARY_VALUE_SIZE = 45

CHART_WIDTH = 585
CHART_HEIGHT = 300
CHART_PADDING = 5
CHART_MARGIN = 1

In [ ]:
# Widgets

In [ ]:
chart_layout = {
    "border": "1px solid black",
    "padding": f"{CHART_PADDING}px",
    "margin": f"{CHART_MARGIN}px",
}

filename_wid = widgets.Text(
    value=FILENAME,
    description="Filename:",
    style={"description_width": f"{TOKEN_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{TOKEN_ENTRY_WIDTH}px"),
)
start_time_wid = widgets.DatetimePicker(
    description="Start:",
    style={"description_width": f"{DATE_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{DATE_ENTRY_WIDTH}px"),
)
end_time_wid = widgets.DatetimePicker(
    description="End:",
    style={"description_width": f"{DATE_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{DATE_ENTRY_WIDTH}px"),
)

refresh_button_wid = widgets.Button(
    description="Refresh", layout=widgets.Layout(width=f"{REFRESH_BUTTON_WIDTH}px")
)

price_out_wid = widgets.Output(layout=chart_layout)
summary_out_wid = widgets.Output(layout=chart_layout)

book_oracle_spread_out_wid = widgets.Output(layout=chart_layout)
book_oracle_spread_quantiles_out_wid = widgets.Output(layout=chart_layout)

book_spread_out_wid = widgets.Output(layout=chart_layout)
book_spread_quantiles_out_wid = widgets.Output(layout=chart_layout)

oracle_spread_out_wid = widgets.Output(layout=chart_layout)
oracle_spread_quantiles_out_wid = widgets.Output(layout=chart_layout)

bid_spread_quantiles_out_wid = widgets.Output(layout=chart_layout)
ask_spread_quantiles_out_wid = widgets.Output(layout=chart_layout)

book_update_delta_quantiles_out_wid = widgets.Output(layout=chart_layout)
oracle_update_delta_quantiles_out_wid = widgets.Output(layout=chart_layout)

log_out_wid = widgets.Output(layout={"width": f"{LOG_WIDTH}px"})

In [ ]:
spacer = widgets.Box(layout=widgets.Layout(height="20px"))

time_range_vbox = widgets.VBox([filename_wid, start_time_wid, end_time_wid])
instrument_vbox = widgets.VBox(
    [refresh_button_wid],
)

left_vbox = widgets.VBox(
    [
        price_out_wid,
        book_oracle_spread_out_wid,
        book_spread_out_wid,
        oracle_spread_out_wid,
        bid_spread_quantiles_out_wid,
        book_update_delta_quantiles_out_wid,
    ]
)
right_vbox = widgets.VBox(
    [
        summary_out_wid,
        book_oracle_spread_quantiles_out_wid,
        book_spread_quantiles_out_wid,
        oracle_spread_quantiles_out_wid,
        ask_spread_quantiles_out_wid,
        oracle_update_delta_quantiles_out_wid,
    ]
)

parameters_hbox = widgets.HBox([time_range_vbox, instrument_vbox, log_out_wid])
charts_hbox = widgets.HBox([left_vbox, right_vbox])

In [ ]:
# Chart Definitions

In [ ]:
price_fig = plot_line(
    pd.Series(),
    [pd.Series(), pd.Series(), pd.Series(), pd.Series()],
    ["Bid", "Ask", "Oracle Bid", "Oracle Ask"],
    title="BBO",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

price_fig = go.FigureWidget(price_fig)

with price_out_wid:
    display(price_fig)

In [ ]:
book_oracle_spread_fig = plot_line(
    pd.Series(),
    [pd.Series()],
    ["Midpoint-Oracle Spread (bps)"],
    title="Midpoint-Oracle Spread (bps)",
    show_legend=False,
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

book_oracle_spread_fig = go.FigureWidget(book_oracle_spread_fig)

with book_oracle_spread_out_wid:
    display(book_oracle_spread_fig)

In [ ]:
book_spread_fig = plot_line(
    pd.Series(),
    [pd.Series()],
    ["Book Spread (bps)"],
    title="Book Spread (bps)",
    show_legend=False,
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

book_spread_fig = go.FigureWidget(book_spread_fig)

with book_spread_out_wid:
    display(book_spread_fig)

In [ ]:
bid_spread_quantiles_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=25,
    name="L1-L2 Bid Spread (bps)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

bid_spread_quantiles_fig = go.FigureWidget(bid_spread_quantiles_fig)

with bid_spread_quantiles_out_wid:
    display(bid_spread_quantiles_fig)

In [ ]:
oracle_spread_fig = plot_line(
    pd.Series(),
    [pd.Series()],
    ["Oracle Spread (bps)"],
    title="Oracle Spread (bps)",
    show_legend=False,
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

oracle_spread_fig = go.FigureWidget(oracle_spread_fig)

with oracle_spread_out_wid:
    display(oracle_spread_fig)

In [ ]:
book_update_delta_quantiles_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=50,
    name="Time between Books (ms)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

book_update_delta_quantiles_fig = go.FigureWidget(book_update_delta_quantiles_fig)

with book_update_delta_quantiles_out_wid:
    display(book_update_delta_quantiles_fig)

In [ ]:
summary_fig = make_subplots(
    rows=1,
    cols=2,
    specs=[
        [{"type": "indicator"}, {"type": "indicator"}],
    ],
)

summary_fig.add_trace(
    go.Indicator(
        mode="number",
        value=0,
        number={"font": {"size": SUMMARY_VALUE_SIZE}},
        title={"text": "Book Updates", "font": {"size": SUMMARY_LABEL_SIZE}},
    ),
    row=1,
    col=1,
)

summary_fig.add_trace(
    go.Indicator(
        mode="number",
        value=0,
        number={
            "font": {"size": SUMMARY_VALUE_SIZE},
        },
        title={"text": "Oracle Updates", "font": {"size": SUMMARY_LABEL_SIZE}},
    ),
    row=1,
    col=2,
)

summary_fig.update_layout(
    width=CHART_WIDTH, height=CHART_HEIGHT, margin=dict(l=0, r=0, t=20, b=0)
)

summary_fig = go.FigureWidget(summary_fig)

with summary_out_wid:
    display(summary_fig)

In [ ]:
book_oracle_spread_quantiles_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=25,
    name="Midpoint-Oracle Spread (bps)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

book_oracle_spread_quantiles_fig = go.FigureWidget(book_oracle_spread_quantiles_fig)

with book_oracle_spread_quantiles_out_wid:
    display(book_oracle_spread_quantiles_fig)

In [ ]:
book_spread_quantiles_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=25,
    name="Book Spread (bps)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

book_spread_quantiles_fig = go.FigureWidget(book_spread_quantiles_fig)

with book_spread_quantiles_out_wid:
    display(book_spread_quantiles_fig)

In [ ]:
ask_spread_quantiles_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=25,
    name="L1-L2 Ask Spread (bps)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

ask_spread_quantiles_fig = go.FigureWidget(ask_spread_quantiles_fig)

with ask_spread_quantiles_out_wid:
    display(ask_spread_quantiles_fig)

In [ ]:
oracle_spread_quantiles_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=25,
    name="Oracle Spread (bps)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

oracle_spread_quantiles_fig = go.FigureWidget(oracle_spread_quantiles_fig)

with oracle_spread_quantiles_out_wid:
    display(oracle_spread_quantiles_fig)

In [ ]:
oracle_update_delta_quantiles_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=50,
    name="Time between Oracle (ms)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

oracle_update_delta_quantiles_fig = go.FigureWidget(oracle_update_delta_quantiles_fig)

with oracle_update_delta_quantiles_out_wid:
    display(oracle_update_delta_quantiles_fig)

In [ ]:
# Update Loop

In [ ]:
books = None
oracle_enabled = False


def update_books():
    global books, oracle_enabled

    try:
        data_path = f"{DATA_DIR}{filename_wid.value}.csv"
        books = pd.read_csv(data_path)

        # oracle_enabled = (
        #     books["oracle_bid", "oracle_bid_size", "oracle_ask", "oracle_ask_size"]
        #     .notna()
        #     .all()
        #     .all()
        # )
        oracle_enabled = True

        books["timestamp_ms"] = pd.to_datetime(books["timestamp_ms"], unit="ms")
        books = books.rename(columns={"timestamp_ms": "timestamp"})
        books["timestamp"] = (
            books["timestamp"].dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
        )

        if start_time_wid.value is not None:
            books = books[books["timestamp"] >= start_time_wid.value]
        if end_time_wid.value is not None:
            books = books[books["timestamp"] <= end_time_wid.value]

        books = books.sort_values(by="timestamp")

        books["midpoint"] = (books["BID1"] + books["ASK1"]) / 2
        books["spread"] = books["ASK1"] - books["BID1"]
        books["spread_bps"] = books["spread"] / books["midpoint"] * 10000

        books["l1_l2_bid_spread"] = books["BID1"] - books["BID2"]
        books["l1_l2_bid_spread_bps"] = (
            books["l1_l2_bid_spread"] / books["midpoint"] * 10000
        )

        books["l1_l2_ask_spread"] = books["ASK2"] - books["ASK1"]
        books["l1_l2_ask_spread_bps"] = (
            books["l1_l2_ask_spread"] / books["midpoint"] * 10000
        )

        if oracle_enabled:
            books["oracle_midpoint"] = (books["oracle_bid"] + books["oracle_ask"]) / 2

            books["oracle_spread"] = books["oracle_ask"] - books["oracle_bid"]
            books["oracle_spread_bps"] = (
                books["oracle_spread"] / books["oracle_midpoint"] * 10000
            )

            books["book_oracle_spread"] = books["midpoint"] - books["oracle_midpoint"]
            books["book_oracle_spread_bps"] = (
                books["book_oracle_spread"] / books["oracle_midpoint"] * 10000
            )

            N = max(
                [
                    int(col[-1])
                    for col in books.columns
                    if col.startswith(("BID", "ASK")) and col[-1].isdigit()
                ]
            )
            bid_columns = [f"BID{i}" for i in range(1, N + 1)] + [
                f"BID_SIZE{i}" for i in range(1, N + 1)
            ]
            ask_columns = [f"ASK{i}" for i in range(1, N + 1)] + [
                f"ASK_SIZE{i}" for i in range(1, N + 1)
            ]
            oracle_columns = [
                "oracle_bid",
                "oracle_bid_size",
                "oracle_ask",
                "oracle_ask_size",
            ]

            books["oracle_change"] = books[oracle_columns].diff().ne(0).any(axis=1)
            books["book_change"] = (
                books[bid_columns + ask_columns].diff().ne(0).any(axis=1)
            )

            oracle_updates = books.loc[books["oracle_change"], "timestamp"]
            oracle_diffs = oracle_updates.diff().dt.total_seconds().mul(1000).dropna()

            book_updates = books.loc[books["book_change"], "timestamp"]
            book_diffs = book_updates.diff().dt.total_seconds().mul(1000).dropna()

    except Exception as e:
        raise e


def update_summary_fig():
    if books is None or len(books) == 0:
        summary_fig.data[0].value = 0
        summary_fig.data[1].value = 0

        return

    num_book_updates = 0
    num_oracle_updates = 0

    if oracle_enabled:
        book_updates = books.loc[books["book_change"], "timestamp"]
        book_diffs = book_updates.diff().dt.total_seconds().mul(1000).dropna()
        num_book_updates = len(book_updates)

        oracle_updates = books.loc[books["oracle_change"], "timestamp"]
        oracle_diffs = oracle_updates.diff().dt.total_seconds().mul(1000).dropna()
        num_oracle_updates = len(oracle_updates)
    else:
        num_book_updates = len(books)

    summary_fig.data[0].value = num_book_updates
    summary_fig.data[1].value = num_oracle_updates


def update_line(fig, x_series, y_series_lst):
    for i in range(len(y_series_lst)):
        fig.data[i].x = x_series
        fig.data[i].y = y_series_lst[i]


def update_price_fig():
    update_line(
        price_fig,
        books["timestamp"],
        [books["BID1"], books["ASK1"], books["oracle_bid"], books["oracle_ask"]],
    )


def update_book_oracle_spread_fig():
    update_line(
        book_oracle_spread_fig,
        books["timestamp"],
        [books["book_oracle_spread_bps"]],
    )


def update_book_spread_fig():
    update_line(book_spread_fig, books["timestamp"], [books["spread_bps"]])


def update_oracle_spread_fig():
    update_line(oracle_spread_fig, books["timestamp"], [books["oracle_spread_bps"]])


def update_quantiles(fig, series, name, round_decimals):
    quantiles = get_quantiles(series, name)

    table_header = dict(values=list(quantiles.columns), align="left")
    table_cells = dict(
        values=[quantiles[k].tolist() for k in quantiles.columns], align="left"
    )

    table_cells["values"][1] = [
        round(v, round_decimals) for v in table_cells["values"][1]
    ]

    fig.data[0].x = series
    fig.data[1].header = table_header
    fig.data[1].cells = table_cells


def update_bid_spread_quantiles_fig():
    update_quantiles(
        bid_spread_quantiles_fig,
        books["l1_l2_bid_spread_bps"],
        "L1-L2 Bid Spread (bps)",
        2,
    )


def update_book_update_delta_quantiles_fig():
    diffs = None

    if oracle_enabled:
        book_updates = books.loc[books["book_change"], "timestamp"]
        diffs = book_updates.diff().dt.total_seconds().mul(1000).dropna()
    else:
        time_diffs = books["timestamp"].diff().dt.total_seconds() * 1000
        diffs = time_diffs.dropna()

    update_quantiles(
        book_update_delta_quantiles_fig, diffs, "Time between Books (ms)", 2
    )


def update_book_oracle_spread_quantiles_fig():
    update_quantiles(
        book_oracle_spread_quantiles_fig,
        books["book_oracle_spread_bps"],
        "Midpoint-Oracle Spread (bps)",
        2,
    )


def update_book_spread_quantiles_fig():
    update_quantiles(book_spread_quantiles_fig, books["spread_bps"], "Spread (bps)", 2)


def update_ask_spread_quantiles_fig():
    update_quantiles(
        ask_spread_quantiles_fig,
        books["l1_l2_ask_spread_bps"],
        "L1-L2 Ask Spread (bps)",
        2,
    )


def update_oracle_spread_quantiles_fig():
    update_quantiles(
        oracle_spread_quantiles_fig,
        books["oracle_spread_bps"],
        "Oracle Spread (bps)",
        2,
    )


def update_oracle_update_delta_quantiles_fig():
    diffs = pd.Series()

    if oracle_enabled:
        oracle_updates = books.loc[books["oracle_change"], "timestamp"]
        diffs = oracle_updates.diff().dt.total_seconds().mul(1000).dropna()

    update_quantiles(
        oracle_update_delta_quantiles_fig, diffs, "Time between Oracle (ms)", 2
    )


def render(*args):
    log_out_wid.outputs = []
    log_out_wid.append_stdout(f"{int(time())}, refreshing every {REFRESH_SEC}s\n")

    try:
        start = time()
        update_books()
        end = time()
        log_out_wid.append_stdout(f"Fetching books took {(end - start):.2f}s\n")
    except Exception as e:
        log_out_wid.append_stdout(f"Failed to fetch new books: {e}\n")
        return

    start = time()

    update_price_fig()
    update_book_spread_fig()
    update_bid_spread_quantiles_fig()
    update_book_update_delta_quantiles_fig()

    update_summary_fig()
    update_book_spread_quantiles_fig()
    update_ask_spread_quantiles_fig()

    if oracle_enabled:
        update_book_oracle_spread_fig()
        update_oracle_spread_fig()
        update_book_oracle_spread_quantiles_fig()
        update_oracle_spread_quantiles_fig()
        update_oracle_update_delta_quantiles_fig()

    end = time()
    log_out_wid.append_stdout(f"Updating charts took {(end - start):.2f}s\n")

In [ ]:
# Start

In [ ]:
for widget in [
    filename_wid,
    start_time_wid,
    end_time_wid,
]:
    widget.observe(render, "value")


def on_refresh_click(b):
    render()


refresh_button_wid.on_click(on_refresh_click)

In [ ]:
display(parameters_hbox)
display(spacer)
display(charts_hbox)


def thread_func(out_wid):
    while True:
        sleep(REFRESH_SEC)
        render()


thread = threading.Thread(target=thread_func, args=(log_out_wid,))
thread.start()

In [ ]:
# for some reason the voila view wont display the first one or two plotly figures
dummy_series = pd.Series(data=[1], index=[1])
plot_quantiles(dummy_series, bins=2, name="", width=10, height=10).show()
plot_quantiles(dummy_series, bins=2, name="", width=10, height=10).show()